# Imports

In [14]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.apps.storymap import JournalStoryMap
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer, Feature
from arcgis.geometry import Geometry, SpatialReference

### Utility Methods

In [55]:
import uuid,json
def getMapAction(fromFeature, forWebmapId, label_field="OBJECTID", label_prefix="Feature Extent"):
    # TODO - make sure input parameters are valid, since we are just inserting them into the mapaction schema
    
    # pull out a value from the feature to display in our anchor tag
    lbl_val = fromFeature.attributes[fromFeature.fields[0]]
    if label_field in fromFeature.fields:
        lbl_val = fromFeature.attributes[label_field]
    
    # format the label
    lbl_frmtd = '{0}: {1}'.format(label_prefix, lbl_val)
    
    # get the geometry extent
    # TODO - Need to validate this or default to a valid extent
    ext = Geometry(fromFeature.geometry).extent
    
    # create a unique id
    # ts = int(round(time.time() * 1000)) this isn't precise enough, gets duplicated in loop
    ts = str(uuid.uuid4())
    #print('id : {0}'.format(ts))
    
    # insert values into our map action schema
    ma = {
        "id":"MJ-ACTION-{0}".format(ts),
        "type": "media",
        "media": {
            "type":"webmap",
            "webmap": {
                "id":forWebmapId,
                "extent": {
                    "xmin": ext[0],
                    "ymin": ext[1],
                    "xmax": ext[2],
                    "ymax": ext[3],
                    "spatialReference":{
                        "wkid": 102100
                    }
                },
                "layers": None,
                "popup": None,
                "overview":{
                    "enable": False,
                    "openByDefault": False
                },
                "legend": {
                    "enable": False,
                    "openByDefault": False
                },
                "geocoder":{
                    "enable": False
                },
                "altText": ""
            }
        }
    }
    
    # create the html string
    content_anchor = '<p><a data-storymaps="{0}" data-storymaps-type="media">{1}</a></p>\n'.format(ma['id'], lbl_frmtd)
    
    # hand back our results to whoever called this function
    return {"content_link": content_anchor, "map_action": ma}

### Connect to Portal

In [10]:
prtl = GIS(url='https://clt-mtorrey10l.esri.com/webmaps', username='siteadmin')

Enter password: ········


### Get a webmap that has some data we want to query

In [16]:
wm = WebMap(prtl.content.get('a01d38d299b741b3ab06d0d50afcc312'))
poly_layer = FeatureLayer(wm.layers[2].url)

### Get the story map

In [37]:
itm = prtl.content.get('bd225ef715c841e1bbd84b6b80426681')
sm = JournalStoryMap(itm)
sections = sm.properties['values']['story']['sections']

True

### Clear out our test section

In [65]:
# clear out any stuff we tried previously
sections[1]['content'] = '' 
sections[1]['contentActions'] = []
sm.save()

True

### Iterate over our features and generate the map actions

In [66]:
poly_features = poly_layer.query()
for f in poly_features.features:
    ma = getMapAction(f, 'a01d38d299b741b3ab06d0d50afcc312')
    sections[1]['content'] += ma['content_link']
    sections[1]['contentActions'].append(ma['map_action'])
   
print('New Section Content :\n{0}'.format(sections[1]['content']))  
sm.save()


New Section Content :
<p><a data-storymaps="MJ-ACTION-773d5418-4343-4fc9-b9b8-150405ef0260" data-storymaps-type="media">Feature Extent: 1</a></p>
<p><a data-storymaps="MJ-ACTION-80551c7e-4e62-432e-a609-ff5fb2014c05" data-storymaps-type="media">Feature Extent: 3</a></p>
<p><a data-storymaps="MJ-ACTION-5b30d936-4cb0-4eac-908f-5190f5634773" data-storymaps-type="media">Feature Extent: 4</a></p>
<p><a data-storymaps="MJ-ACTION-910bdd62-2410-467b-b063-f1d73e4c6424" data-storymaps-type="media">Feature Extent: 2</a></p>



True